In [1]:
from multidimvis_main import *

In [2]:
# Human 
G = nx.read_edgelist('input/ppi_elist.txt',data=False)
organism = 'human'

In [ ]:
# Yeast 
data = pickle.load( open( "input/BIOGRID-ORGANISM-Saccharomyces_cerevisiae_S288c-3.5.185.mitab.pickle", "rb" ) )
organism = 'yeast'


Counter(data['Interaction Detection Method'])
Counter(data['Interaction Types'])

filter_score = data[
                    #(data['Interaction Types'] == 'psi-mi:"MI:0915"(physical association)') +
                    (data['Interaction Types'] == 'psi-mi:"MI:0407"(direct interaction)') 
                    #&
                    #(data['Taxid Interactor A'] == "taxid:559292") & 
                    #(data['Taxid Interactor B'] == "taxid:559292") 
]

g = nx.from_pandas_edgelist(filter_score, '#ID Interactor A', 'ID Interactor B')
g.remove_edges_from(nx.selfloop_edges(g)) #remove self loop

G_cere = g.subgraph(max(nx.connected_components(g), key=len)) # largest connected component (lcc)

# ESSENTIAL GENES 
cere_gene =pd.read_csv("input/Saccharomyces cerevisiae.csv",
           delimiter= ',',
           skipinitialspace=True)
essential_cere = cere_gene[(cere_gene['essentiality status'] == 'E')]
essential_genes_cere_list =  essential_cere['symbols'].tolist()

degree= dict(G_cere.degree())

mg = mygene.MyGeneInfo()
a = mg.querymany(essential_genes_cere_list, scopes='symbol', species=559292)
essential_genes_cere_names = pd.DataFrame.from_dict(a)
essential_genes_cere_entrez =  essential_genes_cere_names['entrezgene'].tolist()

cleaned_entrez_list = [x for x in essential_genes_cere_entrez if str(x) != 'nan']

degree_formatted={}
for k, v in degree.items():
    degree_formatted[k.replace("entrez gene/locuslink:","")] = v
    
index= []
essential = []
for i in cleaned_entrez_list:
    for (key, val) in degree_formatted.items():
        if i==key:
            index.append(key)
            essential.append(val)  

no_essential_cere = cere_gene[(cere_gene['essentiality status'] == 'NE')]
no_essential_genes_cere_list =  no_essential_cere['symbols'].tolist()
b = mg.querymany(no_essential_genes_cere_list, scopes='symbol', species=559292)
no_essential_genes_cere_names = pd.DataFrame.from_dict(b)
no_essential_genes_cere_entrez =  no_essential_genes_cere_names['entrezgene'].tolist()
cleaned_entrez_list_no = [x for x in no_essential_genes_cere_entrez if str(x) != 'nan']

index= []
no_essential = []
for i in cleaned_entrez_list_no:
    for (key, val) in degree_formatted.items():
        if i==key:
            index.append(key)
            no_essential.append(val)
            
#df_cere = pd.DataFrame({'essential': pd.Series(essential), 'no_essential': pd.Series(no_essential)})


no_ess_id = no_essential_genes_cere_names['entrezgene']
ess_id = essential_genes_cere_names['entrezgene']
G = G_cere

#edge_list = nx.write_edgelist(G, "Yeast_edgelist_directinteractiononly.txt")

### GRAPH BASED MATRICES

### Adjacency
+ if two nodes are connected, the matrix value equals 1
+ if two nodes are not connected, the matrix value is 0

In [3]:
%%time

A = nx.adjacency_matrix(G, nodelist=list(G.nodes()))
DM_adj = A.toarray()

CPU times: user 1.14 s, sys: 198 ms, total: 1.34 s
Wall time: 1.34 s


In [4]:
df_adj = pd.DataFrame(DM_adj, columns = list(G.nodes()), index=list(G.nodes()))

In [5]:
df_adj = pd.DataFrame(DM_adj)
df_adj[:10]

,0,1,2,3,4,5,6,7,8,9,...,16366,16367,16368,16369,16370,16371,16372,16373,16374,16375
0,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df_adj.to_csv(r'output_csv/Adjacency_Dataframe_Human.csv', index = True)

### Shortest Path Length
+ = Network distance !!! 

In [ ]:
%%time

d_idx_entz = {}
cc = 0
for entz in sorted(G.nodes()):
    d_idx_entz[cc] = entz
    cc += 1

Mspl = np.zeros(len(list(G.nodes())))

for n1 in range(len(list(G.nodes()))):
    vec = []
    for n2 in range(len(list(G.nodes()))):
        geneA = d_idx_entz[n1]
        geneB = d_idx_entz[n2]
        try:
            spl = nx.shortest_path_length(G,geneA,geneB)
            vec.append(spl)
        except nx.NetworkXNoPath:
            print('no path')
        
    Mspl = np.vstack((Mspl,vec))
Mspl = np.delete(Mspl, (0), axis=0)

DM_spl = Mspl 

In [ ]:
df_spl = pd.DataFrame(DM_spl, columns = list(G.nodes()), index=list(G.nodes()))

In [ ]:
df_spl = pd.DataFrame(DM_spl)
df_spl[:10]

In [ ]:
df_spl.to_csv(r'output_csv/SPL_Dataframe_Human.csv', index = True)

### Random Walk with restart probability
+ the adjacency matrix is the basis for the random walker to serve as information of connections in the graph

In [6]:
# the original function by Felix: 

def rnd_walk_matrix2(A, r, a, num_nodes):
    '''
    Random Walk Operator with restart probability.
    Return Matrix.
    ''' 
    n = num_nodes
    factor = float((1-a)/n) # = 0 if alpha = 1.0 

    E = np.multiply(factor,np.ones([n,n]))              # prepare 2nd scaling term
    A_tele = np.multiply(a,A) + E  
    M = normalize(A_tele, norm='l1', axis=0)                                 # column wise normalized Markov matrix

    # mixture of Markov chains
    del A_tele
    del E

    U = np.identity(n,dtype=int) 
    H = (1-r)*M
    H1 = np.subtract(U,H)
    del U
    del M
    del H    

    W = r*np.linalg.inv(H1)   

    return W

#### VISITING PROBABILITY MATRIX 
+ the values describe the visiting frequency of one node to all others 
+ the higher the more likelihood that the node was visited 
----
+ the restart parameter describes the probability of the walker to return to the initial node (each step)
+ for example, if r = 0.8 the walker will return with a probability of 80 % to the initial node 
+ if he does not return, in a next step, he can choose from all connections outgoing of the node. including the node where he derived from 
+ therefore the diagonal in the visiting probability matrix is r + a small likelihood (e.g. 0.81) of still ending up choosing the initial node 

In [7]:
%%time

# Restart probability
r = 0.8 # originally 0.8
alpha = 0.00006

DM_m = rnd_walk_matrix2(A, r, alpha, len(G.nodes()))

DM_m_df = pd.DataFrame(DM_m)
DM_m_df.sum(axis=1) # axis=1 for columns i.e. features = sum 1

DM_m_df.index=list(G.nodes())
DM_m_df.columns=list(G.nodes())

DM_m_df_T = DM_m_df.T
DM_m_df_T_round = DM_m_df_T.round(6)

CPU times: user 8min 22s, sys: 15.8 s, total: 8min 37s
Wall time: 3min


In [8]:
DM_m_df_T_round

,66008,8473,2561,3759,22906,4928,1994,8481,81610,51361,...,10838,8001,51351,1551,51458,143903,10861,51471,221044,29965
66008,0.800012,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022,...,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012
8473,0.000022,0.800012,0.000012,0.000012,0.000022,0.000012,0.000012,0.000012,0.000012,0.000012,...,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012
2561,0.000022,0.000012,0.800012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,...,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012
3759,0.000022,0.000012,0.000012,0.800012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,...,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012
22906,0.000022,0.000022,0.000012,0.000012,0.800012,0.000022,0.000012,0.000012,0.000012,0.000012,...,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143903,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,...,0.000012,0.000012,0.000012,0.000012,0.000012,0.800012,0.000012,0.000012,0.000012,0.000012
10861,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,...,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.800012,0.000012,0.000012,0.000012
51471,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,...,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.800012,0.000012,0.000012
221044,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,...,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.800012,0.000012


In [9]:
# VISITING PROBABILITY MATRIX 
DM_m_df_T.to_csv(r'_output_csv/RWRvisT_r'+str(r)+'_a'+str(alpha)+'_Dataframe_'+organism+'.csv', index = True)

-----
+ the cosine similarity / distance matrix can either be calculated as input for dimensionality reduction technique 
+ or can be computed with the dim.reduction method, in which case the input shall be a nxm matrix and the metric for calculation of distance shall be chosen in the dim. red. method 

In [15]:
# VISITING PROBABILITY MATRIX 
# TRANSPOSED

DM_m_transposed = DM_m_df.T

DM_m_transposed_ = DM_m_transposed.round(6)
DM_m_transposed_

,66008,8473,2561,3759,22906,4928,1994,8481,81610,51361,...,10838,8001,51351,1551,51458,143903,10861,51471,221044,29965
66008,0.800866,0.004520,0.004456,0.004457,0.004537,0.004513,0.004506,0.004491,0.004454,0.004455,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
8473,0.001233,0.800876,0.000007,0.000007,0.001223,0.000031,0.000122,0.000022,0.000009,0.000007,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2561,0.022916,0.000130,0.801310,0.000130,0.000130,0.000236,0.000142,0.000130,0.000128,0.000128,...,0.0,0.0,0.000001,0.0,0.0,0.000001,0.000001,0.000000,0.000000,0.000000
3759,0.011461,0.000065,0.000065,0.801394,0.000065,0.000096,0.000158,0.000066,0.000064,0.000064,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
22906,0.009608,0.009497,0.000054,0.000054,0.800430,0.009493,0.000104,0.000057,0.000054,0.000056,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143903,0.000000,0.000000,0.000006,0.000000,0.000000,0.000001,0.000032,0.000001,0.000000,0.000000,...,0.0,0.0,0.000250,0.0,0.0,0.800250,0.000250,0.000000,0.000000,0.000000
10861,0.000000,0.000000,0.000006,0.000000,0.000000,0.000001,0.000032,0.000001,0.000000,0.000000,...,0.0,0.0,0.000250,0.0,0.0,0.000250,0.800250,0.000000,0.000000,0.000000
51471,0.000000,0.000001,0.000000,0.000000,0.000000,0.000001,0.000199,0.000001,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.800184,0.000000,0.000000
221044,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000021,0.000002,0.000000,0.000001,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.800108,0.000000


In [16]:
DM_m_transposed.to_csv(r'_output_csv/RWRvis_r'+str(r)+'_transposed_Dataframe_'+organism+'.csv', index = True)

#### DISTANCE MATRIX (based on cosine similarities)
-----
+ calculate the distances from each node to one another 
+ reult = symmetric matrix 
+ 0 indicates close, 1.0 indicates distant

In [36]:
%%time 

metric = "cosine" 
DM_m_new = pd.DataFrame(distance.squareform(distance.pdist(DM_m_transposed, metric)))

CPU times: user 2min, sys: 466 ms, total: 2min
Wall time: 2min


In [39]:
df_m = pd.DataFrame(DM_m_new)#, columns = list(G.nodes()), index=list(G.nodes()))
round(df_m[:10],3)

# DISTANCE MATRIX 
# COSINE BASED 

,0,1,2,3,4,5,6,7,8,9,...,5775,5776,5777,5778,5779,5780,5781,5782,5783,5784
0,0.000,0.973,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,...,1.000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000
1,0.973,0.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,...,1.000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000
2,1.000,1.000,0.000,0.981,0.989,0.998,1.000,1.000,0.996,1.000,...,1.000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000
3,1.000,1.000,0.981,0.000,1.000,1.000,1.000,1.000,1.000,1.000,...,1.000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.996
4,1.000,1.000,0.989,1.000,0.000,1.000,1.000,1.000,1.000,1.000,...,1.000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000
5,1.000,1.000,0.998,1.000,1.000,0.000,0.962,0.958,0.924,0.960,...,1.000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000
6,1.000,1.000,1.000,1.000,1.000,0.962,0.000,0.999,0.998,0.999,...,1.000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000
7,1.000,1.000,1.000,1.000,1.000,0.958,0.999,0.000,0.997,0.999,...,0.993,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000
8,1.000,1.000,0.996,1.000,1.000,0.924,0.998,0.997,0.000,0.997,...,1.000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000
9,1.000,1.000,1.000,1.000,1.000,0.960,0.999,0.999,0.997,0.000,...,1.000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000


In [ ]:
df_m.to_csv(r'output_csv/RWR_'+metric+'_Dataframe_'+organism+'.csv', index = True)

#### -log Markov

In [ ]:
# MARKOV / RWR - LOG

#min_log = lambda t: -np.log(t)
#DM_mlog = np.array([min_log(x/max(x)) for x in DM_m])

In [ ]:
df_mlog = pd.DataFrame(DM_mlog, columns = list(G.nodes()), index=list(G.nodes()))

In [ ]:
df_mlog.to_csv(r'output_csv/RWR_log_'+metric+'_Dataframe_'+organism+'.csv', index = True)